# H20 automl: Porto Seguro (Classification) - Final

Based on: https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_binary_classification_product_backorders.ipynb

Version used for final presentation.

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "AUCPR"  # specify your evaluation metric
stopping = "AUCPR"  # specify your stopping metric
target = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from h2o.automl import H2OAutoML

import h2o
import numpy as np
import pandas as pd

In [3]:
# Import data
train_df = pd.read_csv("porto_train.csv")
test_df = pd.read_csv("porto_test.csv")

In [4]:
train_df.shape

(476170, 60)

In [5]:
test_df.shape

(119042, 59)

In [6]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 10 mins
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,11 months and 2 days !!!
H2O_cluster_name:,H2O_from_python_stever7_49cmhp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.24 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


## Tidy

In [7]:
train_df = train_df.replace(-1, np.nan)
test_df = test_df.replace(-1, np.nan)

## Transform

In [8]:
# Convert categorical variables to "category" data type (after converting to h20 dataframe - easier)
cat_vars = [col for col in train_df.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
train_df = h2o.H2OFrame(train_df)
test_df = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
for col in cat_vars:
    test_df[col] = test_df[col].asfactor()
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_df[col] = train_df[col].asfactor()

In [11]:
predictors = train_df.columns
predictors.remove(target)  # remove "target"
predictors.remove("id")  # remove "id" (should not be a predictor)
predictors.remove("fold")  # remove "fold" (not be a predictor)

## Visualize

(skipped)

## Model

In [12]:
%%time

aml = H2OAutoML(
    max_runtime_secs=time_limit, 
    seed=1, 
    sort_metric=metric,
    stopping_metric=stopping
)

CPU times: user 3.38 ms, sys: 138 µs, total: 3.52 ms
Wall time: 5.15 ms


In [13]:
%%time

aml.train(x=predictors, y=target, training_frame=train_df)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

19:19:29.35: GBM_lr_annealing_selection_AutoML_2_20230821_181949 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

CPU times: user 24.5 s, sys: 2.73 s, total: 27.2 s
Wall time: 1h 4s


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_3_AutoML_2_20230821_181949

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.035012398822582266
RMSE: 0.18711600365169803
LogLoss: 0.14480333438688464
AUC: 0.7960935514319017
AUCPR: 0.1896462392291141
Gini: 0.5921871028638035
Null degrees of freedom: 9852
Residual degrees of freedom: 9805
Null deviance: 3200.8068545728493
Residual deviance: 2853.4945074279485
AIC: 2949.4945074279485

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.061449410129740444
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      8720  756  0.0798   (756.0/9476.0)
1      213   164  0.565    (213.0/377.0)
Total  8933  920  0.0983   (969.0/9853.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0614494    0.252891  148
max f2                       0.0582411    0.349108  157
max f0point5                 0.116196     0.286494  48
max accuracy                 0.224148     0.962245  5
max precision                0.430154     1         0
max recall                   0.0149878    1         380
max specificity              0.430154     1         0
max absolute_mcc             0.0582411    0.236468  157
max min_per_class_accuracy   0.0405891    0.721485  229
max mean_per_class_accuracy  0.0401648    0.723575  231
max tns                      0.430154     9476      0
max fns                      0.430154     376       0
max fps                      0.00953482   9476      399
max tps                      0.0149878    377       380
max tnr                      0.430154     1         0
max fnr                      0.430154     0.997347  0
max fpr                      0.00953482   1         399
max tpr                      0.0149878    1         380

Gains/Lift Table: Avg response rate:  3.83 %, avg score:  3.65 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100477                   0.112149           11.3517    11.3517            0.434343         0.149513   0.434343                    0.149513            0.114058        0.114058                   1035.17   1035.17            0.108149
2        0.0200954                   0.0947308          3.43191    7.3918             0.131313         0.101977   0.282828                    0.125745            0.0344828       0.148541                   243.191   639.18             0.133556
3        0.0300416                   0.0871376          5.60042    6.7987             0.214286         0.0910251  0.260135                    0.11425             0.0557029       0.204244                   460.042   579.87             0.181133
4        0.0400893                   0.0794072          2.11194    5.62405            0.0808081        0.0831302  0.21519                     0.10645             0.0212202       0.225464                   111.194   462.405            0.19275
5        0.0500355                   0.0749848          4.53367    5.4073             0.173469         0.0772495  0.206897                    0.100646            0.0450928       0.270557                   353.367   440.73             0.229295
6        0.100071                    0.0594801          3.65788    4.53259            0.139959         0.0665439  0.173428                    0.0835948           0.183024        0.453581                   265.78

In [14]:
%%time

lb = aml.leaderboard

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 16.5 µs


In [15]:
lb.head(rows=lb.nrows)

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_2_20230821_181949,0.0672009,0.641036,0.151848,0.432692,0.186384,0.0347391
StackedEnsemble_AllModels_4_AutoML_2_20230821_181949,0.0671449,0.641159,0.151842,0.428852,0.186384,0.0347392
StackedEnsemble_BestOfFamily_4_AutoML_2_20230821_181949,0.0664944,0.638724,0.15198,0.433377,0.186416,0.0347509
XGBoost_grid_1_AutoML_2_20230821_181949_model_29,0.0659153,0.636857,0.152117,0.439107,0.186448,0.034763
XGBoost_grid_1_AutoML_2_20230821_181949_model_16,0.0657561,0.63725,0.152101,0.437428,0.186452,0.0347642
XGBoost_grid_1_AutoML_2_20230821_181949_model_23,0.0654968,0.636616,0.152144,0.43116,0.186465,0.0347692
StackedEnsemble_AllModels_2_AutoML_2_20230821_181949,0.0653811,0.637593,0.152095,0.43895,0.186459,0.034767
StackedEnsemble_BestOfFamily_3_AutoML_2_20230821_181949,0.0651974,0.636964,0.152133,0.429679,0.186468,0.0347704
GBM_grid_1_AutoML_2_20230821_181949_model_7,0.0649473,0.634313,0.152286,0.431045,0.186488,0.0347777
GBM_grid_1_AutoML_2_20230821_181949_model_20,0.0648433,0.632125,0.152534,0.435165,0.186529,0.0347929


In [16]:
aml.leader

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_3_AutoML_2_20230821_181949

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.035012398822582266
RMSE: 0.18711600365169803
LogLoss: 0.14480333438688464
AUC: 0.7960935514319017
AUCPR: 0.1896462392291141
Gini: 0.5921871028638035
Null degrees of freedom: 9852
Residual degrees of freedom: 9805
Null deviance: 3200.8068545728493
Residual deviance: 2853.4945074279485
AIC: 2949.4945074279485

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.061449410129740444
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      8720  756  0.0798   (756.0/9476.0)
1      213   164  0.565    (213.0/377.0)
Total  8933  920  0.0983   (969.0/9853.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0614494    0.252891  148
max f2                       0.0582411    0.349108  157
max f0point5                 0.116196     0.286494  48
max accuracy                 0.224148     0.962245  5
max precision                0.430154     1         0
max recall                   0.0149878    1         380
max specificity              0.430154     1         0
max absolute_mcc             0.0582411    0.236468  157
max min_per_class_accuracy   0.0405891    0.721485  229
max mean_per_class_accuracy  0.0401648    0.723575  231
max tns                      0.430154     9476      0
max fns                      0.430154     376       0
max fps                      0.00953482   9476      399
max tps                      0.0149878    377       380
max tnr                      0.430154     1         0
max fnr                      0.430154     0.997347  0
max fpr                      0.00953482   1         399
max tpr                      0.0149878    1         380

Gains/Lift Table: Avg response rate:  3.83 %, avg score:  3.65 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100477                   0.112149           11.3517    11.3517            0.434343         0.149513   0.434343                    0.149513            0.114058        0.114058                   1035.17   1035.17            0.108149
2        0.0200954                   0.0947308          3.43191    7.3918             0.131313         0.101977   0.282828                    0.125745            0.0344828       0.148541                   243.191   639.18             0.133556
3        0.0300416                   0.0871376          5.60042    6.7987             0.214286         0.0910251  0.260135                    0.11425             0.0557029       0.204244                   460.042   579.87             0.181133
4        0.0400893                   0.0794072          2.11194    5.62405            0.0808081        0.0831302  0.21519                     0.10645             0.0212202       0.225464                   111.194   462.405            0.19275
5        0.0500355                   0.0749848          4.53367    5.4073             0.173469         0.0772495  0.206897                    0.100646            0.0450928       0.270557                   353.367   440.73             0.229295
6        0.100071                    0.0594801          3.65788    4.53259            0.139959         0.0665439  0.173428                    0.0835948           0.183024        0.453581                   265.78

In [17]:
perf = aml.leader.model_performance(test_df)
perf

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.034717840052322105
RMSE: 0.18632723915821353
LogLoss: 0.15194910103202394
AUC: 0.635628562854928
AUCPR: 0.06665559684801155
Gini: 0.27125712570985594
Null degrees of freedom: 119041
Residual degrees of freedom: 118994
Null deviance: 37231.77510716302
Residual deviance: 36176.649770108386
AIC: 36272.649770108386

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.05371853700826616
       0       1      Error    Rate
-----  ------  -----  -------  ------------------
0      99017   15690  0.1368   (15690.0/114707.0)
1      3082    1253   0.711    (3082.0/4335.0)
Total  102099  16943  0.1577   (18772.0/119042.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value      idx
---------------------------  -----------  ---------  -----
max f1                       0.0537185    0.117774   196
max f2                       0.0406554    0.198221   250
max f0point5                 0.0918231    0.100728   104
max accuracy                 0.383421     0.963618   3
max precision                0.458991     1          0
max recall                   0.0105656    1          397
max specificity              0.458991     1          0
max absolute_mcc             0.0521117    0.0821172  202
max min_per_class_accuracy   0.0352646    0.59308    276
max mean_per_class_accuracy  0.0394068    0.596909   256
max tns                      0.458991     114707     0
max fns                      0.458991     4334       0
max fps                      0.00937276   114707     399
max tps                      0.0105656    4335       397
max tnr                      0.458991     1          0
max fnr                      0.458991     0.999769   0
max fpr                      0.00937276   1          399
max tpr                      0.0105656    1          397

Gains/Lift Table: Avg response rate:  3.64 %, avg score:  3.66 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100049                   0.111735           3.50464   3.50464            0.127624         0.139187   0.127624                    0.139187            0.0350634       0.0350634                  250.464   250.464            0.0260056
2        0.0200013                   0.0946044          2.93068   3.21778            0.106723         0.102417   0.117178                    0.12081             0.0292964       0.0643599                  193.068   221.778            0.0460349
3        0.0300062                   0.0856353          2.029     2.82141            0.0738875        0.0897856  0.102744                    0.110465            0.0202999       0.0846597                  102.9     182.141            0.056719
4        0.0400027                   0.0794676          2.09993   2.64111            0.0764706        0.0823552  0.0961781                   0.103441            0.0209919       0.105652                   109.993   164.111            0.06813
5        0.0500076                   0.0745248          2.07511   2.52788            0.0755668        0.0768644  0.0920544                   0.0981237           0.0207612       0.126413                   107.511   152.788            0.0792929
6        0.100007                    0.0603665          1.80395   2.16594            0.0656922        0.0665078  0.0788744                   0.0823171           0.0901961       0.216609                   80.3952   116.594            0.121009
7        0.150006                    0.0526106          1.66554   1.99915          

## Communicate

(skipped)